In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from datetime import datetime
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
DEF_NUM_FBINS = 40
DEF_PTA_DUR = 16.03     # [yrs]

# Choose observed GW-Frequency bins based on nyquist sampling
fobs_edges = utils.nyquist_freqs_edges(DEF_PTA_DUR*YR, cad=0.1*YR)
fobs_edges = fobs_edges[:DEF_NUM_FBINS+1]
fobs = utils.midpoints(fobs_edges)

## Basic Example

In [ ]:
gsmf = holo.sam.GSMF_Schechter()               # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()                 # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()                 # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_Standard()    # M-MBulge Relation            (MMB)

sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=40)

In [ ]:
hard_gw = holo.hardening.Hard_GW()
gwb = sam.gwb(fobs, realize=20, hard=hard_gw)    # calculate many different realizations

In [ ]:
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', ylabel='Characteristic Strain $h_c$')

xx = (fobs[1:] + fobs[:-1]) * 0.5
ax.plot(xx, np.median(gwb, axis=1))

ax.fill_between(xx, *np.percentile(gwb, [25, 75], axis=1), alpha=0.2)

plt.show()

In [ ]:
Fixed_Time = holo.hardening.Fixed_Time
fixed = Fixed_Time.from_sam(sam, 2 * GYR)

In [ ]:
gwb_fixed = sam.gwb(fobs, realize=3, hard=fixed)    # calculate many different realizations

In [ ]:
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', ylabel='Characteristic Strain $h_c$')

xx = (fobs[1:] + fobs[:-1]) * 0.5
ax.plot(xx, np.median(gwb_fixed, axis=1))

ax.fill_between(xx, *np.percentile(gwb_fixed, [25, 75], axis=1), alpha=0.2)

plt.show()

## Compare GW-Only to Phenomenological Hardening Model

In [ ]:
sam = holo.sam.Semi_Analytic_Model(shape=20)

In [ ]:
gwb_gw = sam.gwb(fobs_edges, realize=20, hard=holo.hardening.Hard_GW(), zero_stalled=False)

In [ ]:
# hard_times = np.linspace(0.1, 12.0, 3)
hard_times = [0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0, 12.0]
print(hard_times)
ntimes = len(hard_times)
NREALS = 20
nfobs = fobs.size
gwb_ph = np.zeros((ntimes, nfobs, NREALS))

for ii, htime in enumerate(tqdm.tqdm_notebook(hard_times)):
    print(ii, htime)
    hard = holo.hardening.Fixed_Time.from_sam(sam, htime*GYR)
    _gwb = sam.gwb(fobs_edges, realize=NREALS, hard=hard, zero_stalled=True)
    gwb_ph[ii, :, :] = _gwb


In [ ]:
USE_PLAW_REF = False

fig, axes = plot.figax(figsize=[10, 6], ncols=2, bottom=0.25)

xx = fobs * YR
NSAMP = 0

axes[0].set(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN)
axes[1].set(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel="difference")

smap = plot.smap(hard_times, cmap='viridis')
colors = smap.to_rgba(hard_times)

ref = np.median(gwb_gw, axis=-1)
plot.draw_gwb(axes[0], xx, gwb_gw, color='k', nsamp=NSAMP)
axes[1].plot(xx, np.ones_like(ref), color='k', alpha=0.5)

for ii, htime in enumerate(hard_times):
    col = colors[ii]
    plot.draw_gwb(axes[0], xx, gwb_ph[ii, :, :], color=col, nsamp=NSAMP)
    med = np.median(gwb_ph[ii], axis=-1)
    axes[1].plot(xx, med/ref, color=col, alpha=0.5)

cax = fig.add_axes([0.2, 0.08, 0.6, 0.03])
plt.colorbar(smap, label='Lifetime [Gyr]', orientation='horizontal', cax=cax)
plot._twin_hz(axes[0])
plot._twin_hz(axes[1])
plt.show()
filekey = ""
fname = f"phenom-hard{filekey}.png"
fname = Path(holo._PATH_OUTPUT).joinpath(fname)
fig.savefig(fname, dpi=400)
print(f"Saved to {fname}, size {utils.get_file_size(fname)}")
